# Prediction model for ageing

## Dataset analysis

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

2023-01-22 14:03:04.639756: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-22 14:03:04.970516: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-22 14:03:04.970562: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-22 14:03:06.892362: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
dataset = pd.read_csv('./dataset.csv')
dataset.head()

,disease,rsid,status,frequency
0,arthrit,rs11203366,\tassociation,0.56122
1,arthrit,rs2240340,\tassociation,0.56192
2,arthrit,rs28936375,\tPathogenic,0.00017
3,arthrit,rs77245812,\tBenign,0.01614
4,arthrit,rs288326,\trisk_factor,0.08189


In [11]:
dataset.columns

Index(['disease', ' rsid', ' status', ' frequency'], dtype='object')

In [18]:
dataset.dtypes

disease        object
 rsid          object
 status        object
 frequency    float64
dtype: object

In [16]:
dataset.groupby('disease').sum() # 'disease', 'rsid', 'status', 'frequency'

/tmp/ipykernel_31626/4283523074.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dataset.groupby('disease').sum()


,frequency
disease,
arthrit,2.77644
cardiovascular,2.11845
parkinson,0.99861


In [3]:
train, test = train_test_split(dataset, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train samples')
print(len(val), 'validation samples')
print(len(test), 'test samples')

36 train samples
9 validation samples
12 test samples


In [53]:
inputs = {}

for name, column in train.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs


{'disease': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'disease')>,
 ' rsid': <KerasTensor: shape=(None, 1) dtype=string (created by layer ' rsid')>,
 ' status': <KerasTensor: shape=(None, 1) dtype=string (created by layer ' status')>,
 ' frequency': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer ' frequency')>}

In [22]:
dataset_features_dict = {name: np.array(value) 
                         for name, value in train.items()}

In [54]:
def generator():
    for _, row in dataset.iterrows():
        yield (row.values, )

ds = tf.data.Dataset.from_generator(generator, output_types=(tf.float32, tf.string))

In [ ]:
body()

## Build model

In [ ]:
def prediction_model(ds):
  body = tf.keras.Sequential([
    layers.Dense(128,activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(64,activation='relu'),
    layers.Dense(1)
  ])
  result = body(ds)
  model = tf.keras.Model(inputs, result)
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  return model

prediction_model = prediction_model(ds)

## Train model

## Evaluate model (predict rsid)

In [ ]:
model.save('model.h5')